In [ ]:
# 목표 
# 키워드 : 배수구집게
# 범위 : 전체 페이지 
# 수집대상 : 제목, 가격, 판매자(포인트 추가)
# 관련 내용 : 네이버 카페 참조
# 시간차, 프록시 포함 
# if possible: VPN 우회방법까지 

# 시간차란? 
# selenium.wait 
# implicitly wait vs explicitly wait 
# time.sleep 
# https://pythondocs.net/selenium/%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-wait-%EA%B0%9C%EB%85%90-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0-implicitly-wait-vs-explicitly-wait/

# implicitly wait (웹페이지 전체가 넘어올때까지 기다리기) 
# implicitly_wait(10) # 최대 10초 기다리기 (O), 무조건 (X)

# explicity wait (웹페이지의 일부분이 나타날때까지 기다리기)
# 내가 필요한 그 부분이 나올때까지 기다려라 

# time.sleep (물리적인 시간 기다리기)
# time.sleep(10) #무조건 10초 기다리기 

# 프록시란? 
# set IP:port 

In [1]:
# 필요한 모듈과 라이브러리를 로딩
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time, os, math, random, sys, datetime
from datetime import date, timedelta, datetime
import pyautogui
from tqdm import tqdm
import re
from urllib.request import urlopen 

# chromedriver 자동 설치
import chromedriver_autoinstaller

from selenium.webdriver.common.action_chains import ActionChains

In [2]:
print("=" * 80)
print(" 네이버 쇼핑 크롤러 입니다")
print("=" * 80)

f_dir = os.path.expanduser('~') + f'\\Desktop\\crawler-bot\\NaverShopping\\Hasugu\\' # 운영체제에 상관없이 바로 바탕화면 폴더에 작업 

print("=" * 80)
print("데이터 크롤링을 시작합니다")
print("=" * 80)

 네이버 쇼핑 크롤러 입니다
데이터 크롤링을 시작합니다


In [3]:
# 저장될 파일 경로와 이름을 지정 
# 파일을 저장할 폴더 위치를 만들고, 중복일 경우 해당 폴더를 삭제하고 서로 생성합니다 

try:
    os.makedirs(f_dir)
except: pass 
else: 
    os.chdir(f_dir)

fc_name = f_dir + '\\'+f'hasugu.csv' 


In [4]:
# 크롤링 데이터가 들어갈 리스트 생성 
titList = []
priceList = []
sellerList = []
pointList = []

In [5]:
# 프록시 사용 
# https://free-proxy-list.net/ 

# PROXY = "34.121.55.127:8080" # IP:Port 

# webdriver.DesiredCapabilities.CHROME['proxy'] = {
#     "httpProxy": PROXY,
#     "ftpProxy": PROXY, 
#     "sslProxy" : PROXY, 
#     "proxyType" : "MANUAL"
# }

# webdriver.DesiredCapabilities.CHROME['acceptSslCerts']=True

# issue 1) 
# working, 너무 느림

# 인터넷 연결 없음 문제 발생 
#  첫 성공 이후 계속 프록시에 막하는 것 같음 
# 어떻게 풀 수 있죠 ㅠㅠ??? 

In [5]:
# chromedriver-autoinstaller 이용
chromedriver_autoinstaller.install(cwd=True)

# HEADLESS MODE
options = webdriver.ChromeOptions()
# options.add_argument('headless')
options.add_argument('window-size=1920x1080') 
options.add_argument("--disable-gpu")
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
args = ["hide_console", ]

# new options 
# options = webdriver.ChromeOptions()
# #options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# 알림창 끄기
options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 1})

# 웹사이트 접속 후 해당 메뉴로 이동
driver = webdriver.Chrome(options=options,service_args=args)
driver.maximize_window()

# original query 
query = ("https://search.shopping.naver.com/search/all?query=%ED%95%98%EC%88%98%EA%B5%AC%EC%A7%91%EA%B2%8C&cat_id=&frm=NVSHATC")
driver.get(query)

# google.com_test 
# query_test = ("https://www.google com")
# driver.get(query_test)
# driver.get_screenshot_as_file('screenshot.png') # 새로 알게된 옵션 # 화면을 캡처해서 저장 # pdf도 있음 

driver.implicitly_wait(5)
time.sleep(1)

cur_url = driver.current_url 

In [6]:
domain1 = "Naver_Shopping"
print("=============",domain1,"크롤링 진행 중","=============")

============= Naver_Shopping 크롤링 진행 중 =============


In [7]:
# 변수 설정 
page_num = 1
while_code = 0 
cnt = 1 # 총 게시물 건수 카운트 변수 

In [12]:
# 현재 화면의 가장 아래로 스크롤 다운 
# def scroll_down(driver):
#     driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
#     time.sleep(1)
    
# scroll_down(driver)

In [8]:
# messy version    

# issue 1) cnt는 넘어가는데 page 값을 못 넘기고 있다 
# issue 2) item 들을 전혀 못 긁어오고 있다 
# issue 3) item_box가 20개 씩으로 찍힌다 > 원래 40개로 찍혀야함 

while True: 
    
    if while_code == 1: 
        break 
    
    driver.get('https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%ED%95%98%EC%88%98%EA%B5%AC%EC%A7%91%EA%B2%8C&pagingIndex={page_num}&pagingSize=40&productSet=total&query=%ED%95%98%EC%88%98%EA%B5%AC%EC%A7%91%EA%B2%8C&sort=rel&timestamp=&viewType=list')
    driver.implicitly_wait(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    item_box = soup.find('ul', class_= "list_basis").find_all("li", class_=False) 
    print(len(item_box)) # 20개 씩으로 찍힌다 
    
    for item in item_box: 
                                
        try: 
    
            title = item.find('a', class_ = "basicList_link_1MaTN").text
            price = item.find_all('span', class_="price_num_2WUXn").text
            seller = item.find_all('a', class_="basicList_mall__sbVax").text
            point = item.find_all('span', class_= "n_npay_info_2GhW_").text.replace("포인트","").strip()
            
            print(title)
            print(price)
            print(seller)
            print(point)
            print("scout 1")
# #             title = item.find('a', class_ = "basicList_link_1MaTN").text

            titList.append(title)
            priceList.append(price)
            sellerList.append(seller)
            pointList.append(point)
            print("scout 5")    

        except: 
            if page_num > 10: 
                while_code =1 
                break

            pass 
        
    page_num += 1 
    cnt += 40
    print("cnt :", cnt)

    
    
    # Not working 
#         for item in item_box: 
#             title = item.find_all('div', class_= "basicList_title_3P9Q7").text
# #             title = item.find('a', class_ = "basicList_link_1MaTN").text
#             # div class = "basicList_title_3P9Q7"
#             # a target = "_blank" class ="basicList_link_1MaTN".text
            
#             price = item.find_all('div', class_= "basicList_price_are_1UXXR").text
# #             price = item.find_all('span', class_="price_num_2WUXn").text
#             # div class = "basicList_price_are_1UXXR"
#             # span class = "price_num_2WUXn".text()
            
#             seller = item.find_all('a', class_="basicList_mall__sbVax").text
            
#             # a target class = "basicList_mall__sbVax".text()
            
#             point = item.find_all('span', class_= "n_npay_info_2GhW_").text.replace("포인트","").strip()
#             # span class = "n_npay_info_2GhW_"
#             # span # 포인트 57원 .text()
#             # how will I remove point? .text.replace("포인트", "").strip()
            
#             print("scout 3")
            
#             print(title)
#             print(price)
#             print(seller)
#             print(point)
#             print("scout 4")
            
#             titList.append(title)
#             priceList.append(price)
#             sellerList.append(seller)
#             pointList.append(point)
#             print("scout 5")
            
# #     cnt += 40 # 1페이지, 40 포스트 
# #     page_num += 1
# #     print("page_num", cnt//40)
    
# #     if page_num = 5:
# #         while_code = 1
# #         break 
    
#     except: 
# #         page_num = 40 
#         while_code =1 
#         break 
#         # 나가는 조건을 주어야한다 
    
# #     page_num += 1 

# # Just in case 
# # driver.delete_all_cookies()
# # driver.quit()

15
cnt : 41
15
cnt : 81
15
cnt : 121
15
cnt : 161
15
cnt : 201
15
cnt : 241


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=92.0.4515.131)


In [ ]:
#clean code 

In [33]:
# to df #to csv 
# df = pd.DataFrame()

# df['title'] = pd.Series(title)
# df['price'] = pd.Series(price)
# df['seller'] = pd.Series(seller)
# df['point'] = pd.Series(point)

# df.to_csv(fc_name, encoding = "utf-8-sig", index = False)

In [ ]:
e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("1.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("2.파일 저장 완료: csv 파일명 : %s " %fc_name)

driver.quit()
sys.exit("프로그램을 종료합니다.")